In [7]:
#https://geopy.readthedocs.io/en/stable/#installation

import pandas as pd
import numpy as np

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Bay_Wheels_Analysis")

#A rate limiter is needed as the geolocator services time out if all requests sent at once, needs at least one second between each request
from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator, min_delay_seconds=3)



In [2]:
#testing on a coordinate pair from dataset
#result double-checked in Google Maps, the church in output is located very close to coordinates : https://www.google.com/maps/place/37%C2%B047'33.8%22N+122%C2%B014'55.6%22W/@37.7927143,-122.2498766,18z/data=!3m1!4b1!4m5!3m4!1s0x0:0x0!8m2!3d37.7927143!4d-122.2487796

geolocator = Nominatim(user_agent="Bay_Wheels_Analysis")
location = geolocator.reverse("37.7927143, -122.2487796")
print(location.address)


Franklin Elementary School, 10th Avenue, San Antonio, Oakland, Alameda County, California, 94606-3636, United States of America


In [3]:
#loading dataset into python
df = pd.read_excel("Bay_Wheels_Station_Coordinates.xlsx", dtype={"station_name": str, "latitude": float, "longitude": float})

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 3 columns):
station_name    442 non-null object
latitude        442 non-null float64
longitude       442 non-null float64
dtypes: float64(2), object(1)
memory usage: 10.4+ KB


In [8]:
#Applying geolocator.reverse new column 'address' in dataframe
#https://stackoverflow.com/questions/45400288/reverse-geopy-geocoding-pandas


df['address'] = df.apply(lambda row: geolocator.reverse((row['latitude'], row['longitude'])), axis=1)

GeocoderTimedOut: ('Service timed out', 'occurred at index 297')

In [14]:
df.head()

,station_name,latitude,longitude,address
0,10th Ave at E 15th St,37.792714,-122.248780,"(Williams Chapel Baptist Church, 10th Avenue, ..."
1,10th St at Empire St,37.349426,-121.887619,"(North 10th Street at Empire Street, North 10t..."
2,10th St at Fallon St,37.797673,-122.262997,"(OK-L8, 10th Street, Lakeside, Oakland, Califo..."
3,10th St at Mission St,37.355693,-121.892251,"(Hanover Cannery Park, North 10th Street, West..."
4,10th St at University Ave,37.869060,-122.293400,"(1993, Tenth Street, West Berkeley, Berkeley, ..."


In [18]:
#Exported a copy to computer just in case
#https://stackoverflow.com/questions/49608656/saving-a-dataframe-to-csv-file-python
df.to_csv("Bay_Wheels_Station_Address.csv")



In [20]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 4 columns):
station_name    442 non-null object
latitude        442 non-null float64
longitude       442 non-null float64
address         442 non-null object
dtypes: float64(2), object(2)
memory usage: 13.9+ KB


In [35]:
#Convert station_name and address to str to allow filtering of bike stations in San Francisco
#https://stackoverflow.com/questions/41590884/change-data-type-of-a-specific-column-of-a-pandas-dataframe
df['station_name'] = df['station_name'].astype(str)
df['address'] = df['address'].astype(str)
print(df.dtypes)


station_name     object
latitude        float64
longitude       float64
address          object
dtype: object


In [43]:
#Filter dataframe for stations with in address in San Francisco city only 
#https://davidhamann.de/2017/06/26/pandas-select-elements-by-string/
Bay_Wheels_SF_Stations = df[df['address'].str.contains('San Francisco')]

Bay_Wheels_SF_Stations.head(10)

#Export to computer
Bay_Wheels_SF_Stations.to_csv("Bay_Wheels_SF_Station_Address.csv")


,station_name,latitude,longitude,address
5,11th St at Bryant St,37.770030,-122.411726,"SoMa StrEat Food Park, 428, 11th Street, West ..."
6,11th St at Natoma St,37.773507,-122.416040,"pariSoma Innovation Loft, 169, 11th Street, Ci..."
8,14th Ave at Geary Blvd,37.780471,-122.472823,"407, 14th Avenue, Richmond District, San Franc..."
11,14th St at Mission St,37.768265,-122.420110,"Arco, 14th Street, Mission District, San Franc..."
12,15th St at Potrero Ave,37.767079,-122.407359,"Potrero Center, 15th Street, West SoMa, San Fr..."
14,16th St at Prosper St,37.764285,-122.431804,Eureka Valley / Harvey Milk Memorial Branch Li...
15,16th St Depot,37.766349,-122.396292,"971, 16th Street, Mission Bay, San Francisco, ..."
16,16th St Mission BART,37.764710,-122.419957,"Mission District, San Francisco, San Francisco..."
17,16th St Mission BART Station 2,37.764765,-122.420091,"Mission District, San Francisco, San Francisco..."
18,17th & Folsom Street Park (17th St at Folsom St),37.763708,-122.415204,"In Chan Kaajal Park, 17th Street, Mission Dist..."
